In [26]:
import json
import openai
api_key = input("Enter your OpenAI API key: ")
openai.api_key = api_key


training_data = [{
    "prompt": "Where is the billing ->",
    "completion": " You find the billing in the left-hand side menu."
},{
    "prompt":"How do I upgrade my account ->",
    "completion": " Visit you user settings in the left-hand side menu, then click 'upgrade account' button at the top."
}
]

In [27]:
training_data

[{'prompt': 'Where is the billing ->',
  'completion': ' You find the billing in the left-hand side menu.'},
 {'prompt': 'How do I upgrade my account ->',
  'completion': " Visit you user settings in the left-hand side menu, then click 'upgrade account' button at the top."}]

In [28]:
file_name = "training_data.jsonl"

with open(file_name, "w") as output_file:
    for entry in training_data:
        json.dump(entry, output_file)
        output_file.write("\n")

In [29]:
!openai tools fine_tunes.prepare_data -f training_data.jsonl

Analyzing...

- Your file contains 2 prompt-completion pairs. In general, we recommend having at least a few hundred examples. We've found that performance tends to linearly increase for every doubling of the number of examples
- All prompts end with suffix ` ->`
- All completions end with suffix `.`

No remediations found.

You can use your file for fine-tuning:
> openai api fine_tunes.create -t "training_data.jsonl"

After you’ve fine-tuned a model, remember that your prompt has to end with the indicator string ` ->` for the model to start generating completions, rather than continuing with the prompt. Make sure to include `stop=["."]` so that the generated texts ends at the expected place.
Once your model starts training, it'll approximately take 2.47 minutes to train a `curie` model, and less for `ada` and `babbage`. Queue will approximately take half an hour per job ahead of you.


In [30]:
upload_response = openai.File.create(
  file=open(file_name, "rb"),
  purpose='fine-tune'
)
file_id = upload_response.id
upload_response

<File file id=file-U7Z8v8Q1QPbZtiCM5UuJSp85 at 0x281b46fda40> JSON: {
  "object": "file",
  "id": "file-U7Z8v8Q1QPbZtiCM5UuJSp85",
  "purpose": "fine-tune",
  "filename": "file",
  "bytes": 270,
  "created_at": 1687246823,
  "status": "uploaded",
  "status_details": null
}

In [31]:
fine_tune_response = openai.FineTune.create(training_file=file_id)
fine_tune_response

<FineTune fine-tune id=ft-9upEU3J7BlZUeLKxsRtVOJnf at 0x281b46e9ef0> JSON: {
  "object": "fine-tune",
  "id": "ft-9upEU3J7BlZUeLKxsRtVOJnf",
  "hyperparams": {
    "n_epochs": 4,
    "batch_size": null,
    "prompt_loss_weight": 0.01,
    "learning_rate_multiplier": null
  },
  "organization_id": "org-NScQykBF3RzQuZiFmUdxyvyG",
  "model": "curie",
  "training_files": [
    {
      "object": "file",
      "id": "file-U7Z8v8Q1QPbZtiCM5UuJSp85",
      "purpose": "fine-tune",
      "filename": "file",
      "bytes": 270,
      "created_at": 1687246823,
      "status": "uploaded",
      "status_details": null
    }
  ],
  "validation_files": [],
  "result_files": [],
  "created_at": 1687246823,
  "updated_at": 1687246823,
  "status": "pending",
  "fine_tuned_model": null,
  "events": [
    {
      "object": "fine-tune-event",
      "level": "info",
      "message": "Created fine-tune: ft-9upEU3J7BlZUeLKxsRtVOJnf",
      "created_at": 1687246823
    }
  ]
}

In [47]:
fine_tune_events = openai.FineTune.list_events(id=fine_tune_response.id)
fine_tune_events

<OpenAIObject list at 0x2818cb653b0> JSON: {
  "object": "list",
  "data": [
    {
      "object": "fine-tune-event",
      "level": "info",
      "message": "Created fine-tune: ft-9upEU3J7BlZUeLKxsRtVOJnf",
      "created_at": 1687246823
    }
  ]
}

In [42]:
retrieve_response = openai.FineTune.retrieve(id=fine_tune_response.id)
retrieve_response

<FineTune fine-tune id=ft-9upEU3J7BlZUeLKxsRtVOJnf at 0x281b47213b0> JSON: {
  "object": "fine-tune",
  "id": "ft-9upEU3J7BlZUeLKxsRtVOJnf",
  "hyperparams": {
    "n_epochs": 4,
    "batch_size": null,
    "prompt_loss_weight": 0.01,
    "learning_rate_multiplier": null
  },
  "organization_id": "org-NScQykBF3RzQuZiFmUdxyvyG",
  "model": "curie",
  "training_files": [
    {
      "object": "file",
      "id": "file-U7Z8v8Q1QPbZtiCM5UuJSp85",
      "purpose": "fine-tune",
      "filename": "file",
      "bytes": 270,
      "created_at": 1687246823,
      "status": "processed",
      "status_details": null
    }
  ],
  "validation_files": [],
  "result_files": [],
  "created_at": 1687246823,
  "updated_at": 1687246823,
  "status": "pending",
  "fine_tuned_model": null,
  "events": [
    {
      "object": "fine-tune-event",
      "level": "info",
      "message": "Created fine-tune: ft-9upEU3J7BlZUeLKxsRtVOJnf",
      "created_at": 1687246823
    }
  ]
}

In [34]:
# Option 1 | if response.fine_tuned_model != null
fine_tuned_model = fine_tune_response.fine_tuned_model
fine_tuned_model

In [35]:
fine_tuned_model